In [ ]:
!pip install gradio


In [ ]:
import gradio as gr
import cv2
import os
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

# Constants
IMG_SIZE = 128
BATCH_SIZE = 16
EPOCHS = 10
DATASET_PATH = '/content/drive/MyDrive/Training'
CATEGORIES = ['glioma', 'meningioma', 'notumor', 'pituitary']

# Load data
data = []
labels = []

for category in CATEGORIES:
    folder_path = os.path.join(DATASET_PATH, category)
    print(f"Loading {category}...")
    for img in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img)
        try:
            image = cv2.imread(img_path)
            image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
            data.append(image)
            labels.append(category)
        except:
            pass

# Preprocess
data = np.array(data) / 255.0

# Encode labels properly
le = LabelEncoder()
labels_encoded = le.fit_transform(labels)  # Convert to 0-3
labels_categorical = to_categorical(labels_encoded, num_classes=len(CATEGORIES))

# Split
X_train, X_test, y_train, y_test = train_test_split(data, labels_categorical, test_size=0.2, random_state=42)

# Model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(IMG_SIZE, IMG_SIZE, 3)),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    MaxPooling2D((2, 2)),
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(len(CATEGORIES), activation='softmax')
])

model.compile(loss='categorical_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Train
history = model.fit(X_train, y_train, batch_size=BATCH_SIZE, validation_data=(X_test, y_test), epochs=EPOCHS, verbose=1)

# Prediction function
def predict_image(image):
    image = cv2.cvtColor(np.array(image), cv2.COLOR_RGB2BGR)
    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
    image = np.expand_dims(image, axis=0) / 255.0
    prediction = model.predict(image)
    label_index = np.argmax(prediction)
    return CATEGORIES[label_index]

# Gradio UI
interface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type='pil'),
    outputs=gr.Label(num_top_classes=4),
    title="Brain Tumor Classifier",
    description="Upload a brain MRI image to classify it as glioma, meningioma, notumor, or pituitary."
)

interface.launch()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading glioma...
Loading meningioma...
Loading notumor...
Loading pituitary...


/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 105s 748ms/step - accuracy: 0.8246 - loss: 0.5363 - val_accuracy: 0.5259 - val_loss: 2.0540
Epoch 2/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 118s 573ms/step - accuracy: 0.9413 - loss: 0.1647 - val_accuracy: 0.5667 - val_loss: 3.0899
Epoch 3/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 90s 630ms/step - accuracy: 0.9538 - loss: 0.1311 - val_accuracy: 0.5833 - val_loss: 2.3535
Epoch 4/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 83s 616ms/step - accuracy: 0.9562 - loss: 0.1221 - val_accuracy: 0.8426 - val_loss: 0.7185
Epoch 5/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 137s 583ms/step - accuracy: 0.9681 - loss: 0.0844 - val_accuracy: 0.9148 - val_loss: 0.2085
Epoch 6/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 81s 574ms/step - accuracy: 0.9805 - loss: 0.0662 - val_accuracy: 0.6407 - val_loss: 0.8540
Epoch 7/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 87s 616ms/step - accuracy: 0.9793 - loss: 0.0765 - val_accuracy: 0.9574 - val_loss: 0.0973
Epoch 8/10
135/135 ━━━━━━━━━━━━━━━━━━━━ 136s 571ms/step - accuracy: 0.9807 - los